In [1]:
import pandas as pd
import os
import numpy as np

* Cargar archivos

In [19]:
# CARGAR ARCHIVOS PARA MANEJO
def leer_archivos(ruta): 
    archivos = [f for f in os.listdir(ruta) if f.endswith(".csv")]

    dataframes = {}
    errores = {}

    for archivo in archivos:
        path = os.path.join(ruta, archivo)
        try:
            df = pd.read_csv(path, encoding='latin_1', low_memory=False)
            df.columns = df.columns.str.lower()
            dataframes[archivo] = df

        except Exception as e:
            errores[archivo] = str(e)

    # Mostrar archivos con errores
    if errores:
        print("\n⚠️ Archivos con errores al cargar:")
        for archivo, error in errores.items():
            print(f"   ❌ {archivo}: {error}")
    else:
        print("\n✅ Todos los archivos se cargaron correctamente.")
    i = 1
    for archivo in archivos:
        if archivo in dataframes:
            print(f"{i}  📄 {archivo}: {dataframes[archivo].shape[0]} filas, {dataframes[archivo].shape[1]} columnas") 
            i += 1
    
    return dataframes, archivos

In [20]:
def comparar_columnas(dataframes):

    columnas_info = {}
    resultados = {}

    # Extraer columnas y tipos
    for nombre, df in dataframes.items():
        columnas_info[nombre] = {
            "columnas": list(df.columns),
            "tipos": df.dtypes.to_dict()
        }

    # Usar el primer archivo como referencia
    referencia = list(dataframes.keys())[2]
    cols_ref = set(columnas_info[referencia]["columnas"])

    for nombre, info in columnas_info.items():
        if nombre == referencia:
            continue

        cols_actual = set(info["columnas"])
        comunes = cols_ref.intersection(cols_actual)
        distintas = cols_ref.symmetric_difference(cols_actual)

        print(f"\n📂 Comparando {referencia} vs {nombre}:")
        print(f"    ✔️ Columnas comunes ({len(comunes)}): {sorted(comunes)}")
        print(f"    ❌ Columnas distintas ({len(distintas)}): {sorted(distintas)}")

        if len(distintas) == 0:
            print("     ✅ Las columnas son idénticas")
        else:
            print(f"    🔴 Existen {len(distintas)} variables diferentes")


In [70]:
ruta = r"C:\Users\Hp\DENGUE\Data\raw_data\Atlantico"
dataframes, archivos = leer_archivos(ruta)
comparar_columnas(dataframes)


✅ Todos los archivos se cargaron correctamente.
1  📄 Dengue y dengue grave 2017.csv: 1214 filas, 31 columnas
2  📄 Dengue y dengue grave 2018.csv: 3403 filas, 31 columnas
3  📄 Dengue y dengue grave 2019.csv: 4145 filas, 31 columnas
4  📄 Dengue y dengue grave 2020.csv: 2722 filas, 31 columnas
5  📄 Dengue y dengue grave 2021.csv: 7947 filas, 31 columnas
6  📄 Dengue y dengue grave 2022.csv: 10601 filas, 31 columnas
7  📄 Dengue y dengue grave 2023.csv: 2466 filas, 29 columnas

📂 Comparando Dengue y dengue grave 2019.csv vs Dengue y dengue grave 2017.csv:
    ✔️ Columnas comunes (31): ['area', 'año', 'barrio', 'cod_dpto_o', 'cod_mun_o', 'departamento', 'direccion', 'edad', 'fecha_consulta', 'fecha_hospitalizacion', 'fecha_inicio_sintomas', 'fecha_nacimiento', 'fecha_notificacion', 'hospitalizado', 'localidad', 'muerte', 'municipio', 'nombre_evento', 'num_ide_', 'ocupacion_', 'pertencia_etnica', 'pri_ape_', 'pri_nom_', 'regimen_salud', 'seg_ape_', 'seg_nom_', 'semana', 'sexo', 'tip_ide_', 't

In [57]:
# Listado de renombramientos iniciales
changes = {
    'area_': 'area',
    'bar_ver_': 'barrio',
    'con_fin_': 'muerte',
    'dir_res_': 'direccion',
    'edad_': 'edad',
    'fec_con_': 'fecha_consulta',
    'fec_hos_': 'fecha_hospitalizacion', 
    'fec_not': 'fecha_notificacion',
    'fecha_nto_': 'fecha_nacimiento',
    'ini_sin_': 'fecha_inicio_sintomas',
    'localidad_': 'localidad', 
    'ndep_proce': 'departamento',
    'nmun_proce': 'municipio',
    'nom_eve': 'nombre_evento',
    'pac_hos_': 'hospitalizado',
    'per_etn_': 'pertencia_etnica',
    'sexo_': 'sexo',
    'tip_cas_': 'tipo_de_caso', 
    'tip_ss_': 'regimen_salud'
}
  

In [58]:
# Renombrar 
log_renombrado = {}

for archivo in archivos:
    path = os.path.join(ruta, archivo)
    try:
        df = pd.read_csv(path, encoding='latin_1', low_memory=False)
        dataframes[archivo] = df
        df.columns = df.columns.str.lower()
        # Detectar columnas que se pueden renombrar
        columnas_presentes = df.columns.intersection(changes.keys())
        cambios_aplicados = {col: changes[col] for col in columnas_presentes}

        # Renombrar columnas
        df = df.rename(columns=cambios_aplicados)

        # Guardar el archivo sobrescrito
        df.to_csv(path, index=False, encoding='latin1')

        # Registrar cambios
        log_renombrado[archivo] = cambios_aplicados

        print(f"✔️ {archivo}: columnas renombradas → {list(cambios_aplicados.keys())}")

    except Exception as e:
        print(f"❌ Error en {archivo}: {e}")
        log_renombrado[archivo] = f"Error: {str(e)}"

print("\n✅ Renombrado completado. Archivos editados directamente.")


✔️ Dengue y dengue grave 2017.csv: columnas renombradas → []
✔️ Dengue y dengue grave 2018.csv: columnas renombradas → []
✔️ Dengue y dengue grave 2019.csv: columnas renombradas → []
✔️ Dengue y dengue grave 2020.csv: columnas renombradas → []
✔️ Dengue y dengue grave 2021.csv: columnas renombradas → []
✔️ Dengue y dengue grave 2022.csv: columnas renombradas → []
✔️ Dengue y dengue grave 2023.csv: columnas renombradas → ['nom_eve', 'fec_not', 'edad_', 'sexo_', 'ndep_proce', 'nmun_proce', 'area_', 'localidad_', 'bar_ver_', 'dir_res_', 'tip_ss_', 'per_etn_', 'fec_con_', 'ini_sin_', 'tip_cas_', 'pac_hos_', 'fec_hos_', 'con_fin_', 'fecha_nto_']

✅ Renombrado completado. Archivos editados directamente.


In [59]:
dataframes, archivos  = leer_archivos(ruta)
comparar_columnas(dataframes)


✅ Todos los archivos se cargaron correctamente.
1  📄 Dengue y dengue grave 2017.csv: 1214 filas, 31 columnas
2  📄 Dengue y dengue grave 2018.csv: 3403 filas, 31 columnas
3  📄 Dengue y dengue grave 2019.csv: 4145 filas, 31 columnas
4  📄 Dengue y dengue grave 2020.csv: 2722 filas, 31 columnas
5  📄 Dengue y dengue grave 2021.csv: 7947 filas, 31 columnas
6  📄 Dengue y dengue grave 2022.csv: 10601 filas, 31 columnas
7  📄 Dengue y dengue grave 2023.csv: 6682 filas, 134 columnas

📂 Comparando Dengue y dengue grave 2019.csv vs Dengue y dengue grave 2017.csv:
    ✔️ Columnas comunes (31): ['area', 'año', 'barrio', 'cod_dpto_o', 'cod_mun_o', 'departamento', 'direccion', 'edad', 'fecha_consulta', 'fecha_hospitalizacion', 'fecha_inicio_sintomas', 'fecha_nacimiento', 'fecha_notificacion', 'hospitalizado', 'localidad', 'muerte', 'municipio', 'nombre_evento', 'num_ide_', 'ocupacion_', 'pertencia_etnica', 'pri_ape_', 'pri_nom_', 'regimen_salud', 'seg_ape_', 'seg_nom_', 'semana', 'sexo', 'tip_ide_', '

### Eliminar variables 

In [60]:
delete = [
    'ajuste_', 'cbmte_', 'cen_pobla_', 'cer_def_', 'cod_ase_', 
    'cod_dpto_r', 'cod_mun_r', 'cod_pais_o', 'cod_pre', 'cod_sub', 
    'eve_historico', 'fec_aju_', 'fec_arc_xl', 'fec_def_', 
    'fm_fuerza', 'fm_grado', 'fm_unidad', 
    'gp_carcela', 'gp_desmovi', 'gp_desplaz', 'gp_discapa', 'gp_gestan', 
    'gp_indigen', 'gp_mad_com', 'gp_migrant', 'gp_otros', 'gp_pobicbf', 
    'gp_psiquia', 'gp_vic_vio', 'ndep_notif', 'ndep_resi', 'nit_upgd', 
    'nmun_notif', 'nmun_resi', 'nom_dil_f_', 'nom_upgd', 
    'tel_dil_f_', 'telefono_', 'vereda_', 'daño_organ', 'muesriñon', 'ndep_proce.1', 'nmun_proce.1', 'sem_ges']

In [61]:
# Eliminar archivos
log_eliminacion = {}

for archivo in archivos:
    path_csv = os.path.join(ruta, archivo)
    try:
        df = pd.read_csv(path_csv, encoding='latin1', low_memory=False)
        df.columns = df.columns.str.lower()
        # Detectar columnas presentes que se van a eliminar
        cols_a_eliminar = [c for c in delete if c in df.columns]

        # Eliminar columnas
        df = df.drop(columns=cols_a_eliminar, errors="ignore")

        # Guardar sobrescribiendo el archivo
        df.to_csv(path_csv, index=False, encoding='latin1')

        # Registrar cambios
        log_eliminacion[archivo] = cols_a_eliminar
        print(f"✔️ {archivo}: columnas eliminadas → {cols_a_eliminar}")

    except Exception as e:
        log_eliminacion[archivo] = f"Error: {str(e)}"
        print(f"❌ Error en {archivo}: {e}")

print("\n✅ Eliminación completada. Archivos editados directamente.")

✔️ Dengue y dengue grave 2017.csv: columnas eliminadas → []
✔️ Dengue y dengue grave 2018.csv: columnas eliminadas → []
✔️ Dengue y dengue grave 2019.csv: columnas eliminadas → []
✔️ Dengue y dengue grave 2020.csv: columnas eliminadas → []
✔️ Dengue y dengue grave 2021.csv: columnas eliminadas → []
✔️ Dengue y dengue grave 2022.csv: columnas eliminadas → []
✔️ Dengue y dengue grave 2023.csv: columnas eliminadas → ['ajuste_', 'cbmte_', 'cen_pobla_', 'cer_def_', 'cod_ase_', 'cod_pais_o', 'cod_pre', 'cod_sub', 'fec_aju_', 'fec_arc_xl', 'fec_def_', 'fm_fuerza', 'fm_grado', 'fm_unidad', 'gp_carcela', 'gp_desmovi', 'gp_desplaz', 'gp_discapa', 'gp_gestan', 'gp_indigen', 'gp_mad_com', 'gp_migrant', 'gp_otros', 'gp_pobicbf', 'gp_psiquia', 'gp_vic_vio', 'ndep_notif', 'ndep_resi', 'nit_upgd', 'nmun_notif', 'nmun_resi', 'nom_dil_f_', 'nom_upgd', 'tel_dil_f_', 'telefono_', 'vereda_', 'ndep_proce.1', 'nmun_proce.1', 'sem_ges']

✅ Eliminación completada. Archivos editados directamente.


In [62]:
dataframes, archivos  = leer_archivos(ruta)
comparar_columnas(dataframes)


✅ Todos los archivos se cargaron correctamente.
1  📄 Dengue y dengue grave 2017.csv: 1214 filas, 31 columnas
2  📄 Dengue y dengue grave 2018.csv: 3403 filas, 31 columnas
3  📄 Dengue y dengue grave 2019.csv: 4145 filas, 31 columnas
4  📄 Dengue y dengue grave 2020.csv: 2722 filas, 31 columnas
5  📄 Dengue y dengue grave 2021.csv: 7947 filas, 31 columnas
6  📄 Dengue y dengue grave 2022.csv: 10601 filas, 31 columnas
7  📄 Dengue y dengue grave 2023.csv: 6682 filas, 95 columnas

📂 Comparando Dengue y dengue grave 2019.csv vs Dengue y dengue grave 2017.csv:
    ✔️ Columnas comunes (31): ['area', 'año', 'barrio', 'cod_dpto_o', 'cod_mun_o', 'departamento', 'direccion', 'edad', 'fecha_consulta', 'fecha_hospitalizacion', 'fecha_inicio_sintomas', 'fecha_nacimiento', 'fecha_notificacion', 'hospitalizado', 'localidad', 'muerte', 'municipio', 'nombre_evento', 'num_ide_', 'ocupacion_', 'pertencia_etnica', 'pri_ape_', 'pri_nom_', 'regimen_salud', 'seg_ape_', 'seg_nom_', 'semana', 'sexo', 'tip_ide_', 't

In [63]:
delete = [
    'acum_liqui', 'agente', 'artralgia', 'aum_hemato', 'caida_plaq', 'cefalea', 
    'choque', 'clasfinal', 'clasificacion', 'cod_dep_d', 'cod_eve', 'cod_mun_d', 
    'cod_pais_d', 'cod_pais_r', 'conducta', 'control', 'daã±o_organ', 'desplazami', 
    'diarrea', 'direclabor', 'dolor_abdo', 'dolrretroo', 'erupcionr', 'estrato_', 
    'extravasac', 'famantdngu', 'fec_exa', 'fec_exp', 'fec_rec', 'fechahora', 'fiebre', 
    'fuente_', 'hem_mucosa', 'hemorr_hem', 'hepatomeg', 'hipotensio', 'hipotermia', 
    'iden_gener', 'lab_ajuste', 'lat_dir', 'long_dir', 'malgias', 'muesbazo', 'muescerebr', 
    'mueshigado', 'muesmedula', 'muesmiocar', 'muespulmon', 'muesriã±on', 'muestra', 'muesttejid', 
    'nacionali_', 'ndep_resi.1', 'nmun_notif.1', 'nmun_resi.1', 'nom_eve.1', 'nom_grupo_', 
    'nombre', 'nombre_nacionalidad', 'npais_proce', 'npais_resi', 'num_ide_.1', 'nuni_modif', 
    'orient_sex', 'otra_ident', 'otra_orien', 'prueba', 'resultado', 'sem_ges_', 'somnolenci',
    'uni_modif', 'valor', 'version', 'vomito']
   

In [64]:
# Eliminar archivos
log_eliminacion = {}

for archivo in archivos:
    path_csv = os.path.join(ruta, archivo)
    try:
        df = pd.read_csv(path_csv, encoding='latin1', low_memory=False)
        df.columns = df.columns.str.lower()
        # Detectar columnas presentes que se van a eliminar
        cols_a_eliminar = [c for c in delete if c in df.columns]

        # Eliminar columnas
        df = df.drop(columns=cols_a_eliminar, errors="ignore")

        # Guardar sobrescribiendo el archivo
        df.to_csv(path_csv, index=False, encoding='latin1')

        # Registrar cambios
        log_eliminacion[archivo] = cols_a_eliminar
        print(f"✔️ {archivo}: columnas eliminadas → {cols_a_eliminar}")

    except Exception as e:
        log_eliminacion[archivo] = f"Error: {str(e)}"
        print(f"❌ Error en {archivo}: {e}")

print("\n✅ Eliminación completada. Archivos editados directamente.")

✔️ Dengue y dengue grave 2017.csv: columnas eliminadas → []
✔️ Dengue y dengue grave 2018.csv: columnas eliminadas → []
✔️ Dengue y dengue grave 2019.csv: columnas eliminadas → []
✔️ Dengue y dengue grave 2020.csv: columnas eliminadas → []
✔️ Dengue y dengue grave 2021.csv: columnas eliminadas → []
✔️ Dengue y dengue grave 2022.csv: columnas eliminadas → []
✔️ Dengue y dengue grave 2023.csv: columnas eliminadas → ['acum_liqui', 'agente', 'artralgia', 'aum_hemato', 'caida_plaq', 'cefalea', 'choque', 'clasfinal', 'cod_dep_d', 'cod_mun_d', 'cod_pais_d', 'cod_pais_r', 'conducta', 'control', 'daã±o_organ', 'desplazami', 'diarrea', 'direclabor', 'dolor_abdo', 'dolrretroo', 'erupcionr', 'estrato_', 'extravasac', 'famantdngu', 'fec_exa', 'fec_exp', 'fec_rec', 'fiebre', 'fuente_', 'hem_mucosa', 'hemorr_hem', 'hepatomeg', 'hipotensio', 'hipotermia', 'lab_ajuste', 'malgias', 'muesbazo', 'muescerebr', 'mueshigado', 'muesmedula', 'muesmiocar', 'muespulmon', 'muesriã±on', 'muestra', 'muesttejid', 'n

In [65]:
dataframes, archivos  = leer_archivos(ruta)
comparar_columnas(dataframes)


✅ Todos los archivos se cargaron correctamente.
1  📄 Dengue y dengue grave 2017.csv: 1214 filas, 31 columnas
2  📄 Dengue y dengue grave 2018.csv: 3403 filas, 31 columnas
3  📄 Dengue y dengue grave 2019.csv: 4145 filas, 31 columnas
4  📄 Dengue y dengue grave 2020.csv: 2722 filas, 31 columnas
5  📄 Dengue y dengue grave 2021.csv: 7947 filas, 31 columnas
6  📄 Dengue y dengue grave 2022.csv: 10601 filas, 31 columnas
7  📄 Dengue y dengue grave 2023.csv: 6682 filas, 32 columnas

📂 Comparando Dengue y dengue grave 2019.csv vs Dengue y dengue grave 2017.csv:
    ✔️ Columnas comunes (31): ['area', 'año', 'barrio', 'cod_dpto_o', 'cod_mun_o', 'departamento', 'direccion', 'edad', 'fecha_consulta', 'fecha_hospitalizacion', 'fecha_inicio_sintomas', 'fecha_nacimiento', 'fecha_notificacion', 'hospitalizado', 'localidad', 'muerte', 'municipio', 'nombre_evento', 'num_ide_', 'ocupacion_', 'pertencia_etnica', 'pri_ape_', 'pri_nom_', 'regimen_salud', 'seg_ape_', 'seg_nom_', 'semana', 'sexo', 'tip_ide_', 't

****

In [72]:
dataframes, archivos = leer_archivos(ruta)

# Unir todos en un solo DataFrame
df_final = pd.concat(dataframes.values(), ignore_index=True)


✅ Todos los archivos se cargaron correctamente.
1  📄 Dengue y dengue grave 2017.csv: 1214 filas, 31 columnas
2  📄 Dengue y dengue grave 2018.csv: 3403 filas, 31 columnas
3  📄 Dengue y dengue grave 2019.csv: 4145 filas, 31 columnas
4  📄 Dengue y dengue grave 2020.csv: 2722 filas, 31 columnas
5  📄 Dengue y dengue grave 2021.csv: 7947 filas, 31 columnas
6  📄 Dengue y dengue grave 2022.csv: 10601 filas, 31 columnas
7  📄 Dengue y dengue grave 2023.csv: 2466 filas, 29 columnas


In [73]:
df_final = df_final[sorted(df_final.columns)]
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32498 entries, 0 to 32497
Data columns (total 31 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   area                   32498 non-null  int64  
 1   año                    32498 non-null  int64  
 2   barrio                 32374 non-null  object 
 3   cod_dpto_o             30032 non-null  float64
 4   cod_mun_o              30032 non-null  float64
 5   departamento           32498 non-null  object 
 6   direccion              32498 non-null  object 
 7   edad                   32498 non-null  int64  
 8   fecha_consulta         32498 non-null  object 
 9   fecha_hospitalizacion  32498 non-null  object 
 10  fecha_inicio_sintomas  32498 non-null  object 
 11  fecha_nacimiento       32498 non-null  object 
 12  fecha_notificacion     32498 non-null  object 
 13  hospitalizado          32498 non-null  int64  
 14  localidad              31045 non-null  object 
 15  mu

In [74]:
count_duplicados = df_final.duplicated().sum()
print(f"Número de filas duplicadas exactas en columnas de interés: {count_duplicados}")

Número de filas duplicadas exactas en columnas de interés: 0


In [75]:
df_final['nombre_evento'] = df_final['nombre_evento'].str.strip()
df_final['nombre_evento'].unique()

array(['DENGUE', 'DENGUE GRAVE'], dtype=object)

In [76]:
# Crear evento a partir del codigo 
# evento = Clasifico if code_eve = 210 
# evento = Grave if code_eve = 220
df_final['evento'] = np.where(df_final['nombre_evento'] == 'DENGUE', 'Clasico', 
                       np.where(df_final['nombre_evento'] == 'DENGUE GRAVE', 'Grave', 'Otros'))
df_final['evento'].unique()

array(['Clasico', 'Grave'], dtype=object)

In [77]:
df_final.columns

Index(['area', 'año', 'barrio', 'cod_dpto_o', 'cod_mun_o', 'departamento',
       'direccion', 'edad', 'fecha_consulta', 'fecha_hospitalizacion',
       'fecha_inicio_sintomas', 'fecha_nacimiento', 'fecha_notificacion',
       'hospitalizado', 'localidad', 'muerte', 'municipio', 'nombre_evento',
       'num_ide_', 'ocupacion_', 'pertencia_etnica', 'pri_ape_', 'pri_nom_',
       'regimen_salud', 'seg_ape_', 'seg_nom_', 'semana', 'sexo', 'tip_ide_',
       'tipo_de_caso', 'uni_med_', 'evento'],
      dtype='object')

In [78]:
delete =['num_ide_', 'ocupacion_', 'pri_ape_', 'pri_nom_',
        'seg_ape_', 'seg_nom_', 'tip_ide_']

In [79]:
df_final = df_final.drop(columns=delete, errors="ignore")
df_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32498 entries, 0 to 32497
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   area                   32498 non-null  int64  
 1   año                    32498 non-null  int64  
 2   barrio                 32374 non-null  object 
 3   cod_dpto_o             30032 non-null  float64
 4   cod_mun_o              30032 non-null  float64
 5   departamento           32498 non-null  object 
 6   direccion              32498 non-null  object 
 7   edad                   32498 non-null  int64  
 8   fecha_consulta         32498 non-null  object 
 9   fecha_hospitalizacion  32498 non-null  object 
 10  fecha_inicio_sintomas  32498 non-null  object 
 11  fecha_nacimiento       32498 non-null  object 
 12  fecha_notificacion     32498 non-null  object 
 13  hospitalizado          32498 non-null  int64  
 14  localidad              31045 non-null  object 
 15  mu

In [80]:
df_final.columns

Index(['area', 'año', 'barrio', 'cod_dpto_o', 'cod_mun_o', 'departamento',
       'direccion', 'edad', 'fecha_consulta', 'fecha_hospitalizacion',
       'fecha_inicio_sintomas', 'fecha_nacimiento', 'fecha_notificacion',
       'hospitalizado', 'localidad', 'muerte', 'municipio', 'nombre_evento',
       'pertencia_etnica', 'regimen_salud', 'semana', 'sexo', 'tipo_de_caso',
       'uni_med_', 'evento'],
      dtype='object')

In [81]:
df_final['departamento'].value_counts()

departamento
ATLANTICO            31800
MAGDALENA              255
GUAJIRA                129
BOLIVAR                 82
47                      62
CESAR                   25
DEPTO DESCONOCIDO       23
SUCRE                   19
44                      17
CORDOBA                 11
13                      10
CUNDINAMARCA             8
ANTIOQUIA                7
20                       6
VALLE                    5
SANTANDER                5
SAN ANDRES               5
AMAZONAS                 3
70                       3
5                        3
NORTE SANTANDER          3
BOYACA                   2
TOLIMA                   2
CALDAS                   2
GUAVIARE                 2
CASANARE                 1
NARIÑO                   1
CAQUETA                  1
CHOCO                    1
54                       1
CQ                       1
23                       1
52                       1
68                       1
Name: count, dtype: int64

In [82]:
# Filtrar solo los registros donde el departamento es ATLANTICO
df_final = df_final[df_final['departamento'] == "ATLANTICO"]

print(df_final.shape)  # para ver cuántos registros quedaron


(31800, 25)


In [83]:
delete =['barrio', 'cod_dpto_o', 'cod_mun_o', 'direccion', 'localidad', 'departamento']
df_final = df_final.drop(columns=delete, errors="ignore")

In [84]:
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 31800 entries, 0 to 32497
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   area                   31800 non-null  int64 
 1   año                    31800 non-null  int64 
 2   edad                   31800 non-null  int64 
 3   fecha_consulta         31800 non-null  object
 4   fecha_hospitalizacion  31800 non-null  object
 5   fecha_inicio_sintomas  31800 non-null  object
 6   fecha_nacimiento       31800 non-null  object
 7   fecha_notificacion     31800 non-null  object
 8   hospitalizado          31800 non-null  int64 
 9   muerte                 31800 non-null  int64 
 10  municipio              31800 non-null  object
 11  nombre_evento          31800 non-null  object
 12  pertencia_etnica       31800 non-null  int64 
 13  regimen_salud          31800 non-null  object
 14  semana                 31800 non-null  int64 
 15  sexo                   3

In [85]:
df = df_final

In [86]:
# Reemplazar el valor específico
df["municipio"] = df["municipio"].replace(
    "* ATLANTICO. MUNICIPIO DESCONO", "desconocido"
)

# Verificar
print(df["municipio"].unique())

['BARRANQUILLA' 'SOLEDAD' 'REPELON' 'JUAN DE ACOSTA' 'GALAPA' 'USIACURI'
 'BARANOA' 'POLO NUEVO' 'SABANALARGA' 'SABANAGRANDE' 'MALAMBO' 'LURUACO'
 'CAMPO DE LA CRUZ' 'SANTO TOMAS' 'desconocido' 'TUBARA' 'PONEDERA'
 'PUERTO COLOMBIA' 'MANATI' 'CANDELARIA' 'PALMAR DE VARELA' 'SUAN' 'PIOJO'
 'SANTA LUCIA']


In [59]:
df = df.drop(columns=["departamento", 'cod_dpto_o'], errors="ignore")
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32594 entries, 0 to 33185
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   area                   32594 non-null  int64 
 1   año                    32594 non-null  int64 
 2   edad                   32594 non-null  int64 
 3   fecha_consulta         32594 non-null  object
 4   fecha_hospitalizacion  32594 non-null  object
 5   fecha_inicio_sintomas  32594 non-null  object
 6   fecha_nacimiento       32594 non-null  object
 7   fecha_notificacion     32594 non-null  object
 8   hospitalizado          32594 non-null  int64 
 9   muerte                 32594 non-null  int64 
 10  municipio              32594 non-null  object
 11  nombre_evento          32594 non-null  object
 12  pertencia_etnica       32594 non-null  int64 
 13  regimen_salud          32594 non-null  object
 14  semana                 32594 non-null  int64 
 15  sexo                   3

In [87]:
import os

ruta_salida = r"C:\Users\Hp\DENGUE\Data"
archivo = "data_atlantico.csv"

# Crear ruta completa
path_out = os.path.join(ruta_salida, archivo)

# Guardar
df_final.to_csv(path_out, index=False, encoding="latin1")
print(f"✔️ Guardado en: {path_out}")

✔️ Guardado en: C:\Users\Hp\DENGUE\Data\data_atlantico.csv
